# 数据拆分

首先，激活运行环境（vsearch、fastqc、biopython都需要）

In [ ]:
conda activate /home/LDlab/BioSoft/anaconda3/envs/qiime2-2020.2

### 步骤一：双端合并：

```shell
vsearch --fastq_mergepairs FILENAME-1 --reverse FILENAME-2 --fastqout FILENAME-out --fastqout_notmerged_fwd FILENAME-a --fastqout_notmerged_rev FILENAME-b 1>log—file 2>>log—file
```

实际操作例子：在/home/yxtan/qzp_16S_pools/4-22-batch   
```shell
vsearch --fastq_mergepairs /home/LDlab/quzepeng/已知OD之比测序数据-20200421/QZP1_S1_L001_R1_001.fastq.gz --reverse /home/LDlab/quzepeng/已知OD之比测序数据-20200421/QZP1_S1_L001_R2_001.fastq.gz --fastqout qzp_merged.fq --fastqout_notmerged_fwd qzp_unmapped_1.fq --fastqout_notmerged_rev qzp_unmapped_2.fq 1>vserach_log_qzp.txt 2>>vserach_log_qzp.txt
```

### 步骤二：拆分样品（根据引物序列的完美匹配）

```shell
python /home/yxtan/qiime2_custom_scripts/parse_sample_perfectmatch.py -i 合并后的fq路径 -t 每个样品对应的barcode序列 -f 正向引物长度 -r 反向引物长度
```

实际操作例子：在/home/yxtan/qzp_16S_pools/4-22-batch   
从20200420样品信息.xlsx中提取B、D、E三列就是sample_indexes.txt 【为了后续画图等方便，其实B列的数字最好前面用0来补全N位数，这个可以在excel里面弄】（但是一定要删干净，不然会说有不对应的column数；且最后一行不能有回车，也就是不能最后有一行空的行）

```shell
python /home/yxtan/qiime2_custom_scripts/parse_sample_perfectmatch.py -i qzp_merged.fq -t sample_indexes.txt -f 20 -r 18
```

In [ ]:
"/home/LDlab/BioSoft/Scripts/parse_sample_perfectmatch.py"

### 步骤三：拆分样品（根据引物序列的完美匹配）

```shell
bash /home/yxtan/qzp_16S_pools/data/Bowtie2align_and_feature_extract.sh input_folder input_ref.fa input_ref.bed
```

实际操作例子：在/home/yxtan/qzp_16S_pools/4-22-batch   
22株根际菌.fa可以直接用1-22号菌V5-V7序列信息(删除-).txt的内容   
22株根际菌.bed需要从1-22号菌V5-V7序列信息(删除-).txt里面，通过excel里面的len命令生成每个序列对应的长度，从而生成（我没专门写脚本）

```shell
bash /home/yxtan/qzp_16S_pools/data/Bowtie2align_and_feature_extract.sh parse 22株根际菌.fa 22株根际菌.bed
```

In [ ]:
"/home/LDlab/BioSoft/Scripts/Bowtie2align_and_feature_extract.sh"

In [ ]:
#input1: the folder of fastqs
#input2: The fasta file of the refernce region of each strain; the name of strains should be the same as the ones in the bed file
#input3: The bed file (must be customized into strains in specific format) and the fasta with bowtie2 index of the refernce strains

#output_path: current folder
#final output: count_matrix.txt （主要的结果文件）
#intermediate outputs: intermediate folder with sam\bam\bai of each sample.

吴璐师姐数据拆分   
1.	join_paired_ends.py 序列拼接

```shell
join_paired_ends.py [options] {-f/--forward_reads_fp FORWARD_READS_FP -r/--reverse_reads_fp REVERSE_READS_FP -o/--output_dir OUTPUT_DIR}
```   
2.	extract_barcodes.py barcodes提取
```shell
extract_barcodes.py -f inseqs.fastq -c barcode_single_end --bc1_len 7-o processed_seqs
```

3.	制作和验证mapping文件

SampleID BarcodeSequence LinkerPrimerSequence Treatment DOB Description
可利用模版在excel中修改，然后另存为txt 制表符分开的格式 Tab-delimited .txt
然后可以check_id_map.py 检查mapping文件是否正确

4.	样品分割 split_libraries_fastq.py
```shell
split_libraries_fastq.py -i read.fastq -b barcode.fastq –-barcode_type 7-o slout_q20/ -m map.txt -q 19
```

```shell
qiime > split_libraries_fastq.py -i processing_seqs/reads.fastq -b processing_seqs/barcodes.fastq --barcode_type 7 –o slout_q20/ -m 1_raw_data_group1/mapping_file.txt -q 19
```